In [ ]:
import mitsuba as mi
mi.set_variant("cuda_ad_rgb")

import numpy as np
import matplotlib.pyplot as plt

import import_ipynb
import importlib

import testing_scene as ts
importlib.reload(ts)

import vapl_utils as utils
importlib.reload(utils)

import torch

# TODO: add visualization graph
# from torchviz import make_dot
# import torchvision
# from torchview import draw_graph

import config

In [ ]:
loss_function = utils.Loss(utils.weighted_loss)
field = utils.vapl_grid(ts.scene.bbox().min, ts.scene.bbox().max, 3, 4, 8).cuda()
rhs_integrator = utils.RHSIntegrator(field, loss_function)

def should_render(epoch):
    if epoch < 50:
        return epoch % 5 == 0
    elif epoch < 500:
        return epoch % 20 == 0
    elif epoch < 2000:
        return epoch % 100 == 0
    else:
        return epoch % 250 == 0

for epoch in range(10001):
    rhs_integrator.epoch = epoch

    rhs_image = mi.render(ts.scene, spp=1, integrator=rhs_integrator)

    if (should_render(epoch)):
        with torch.no_grad():
            gaussians, vmfs = utils.get_all_gaussians(field)

            mean = gaussians[:, :3]
            utils.print_tensor_stats(mean, "positions")
            variance = gaussians[:, 3]
            amplitude = vmfs[:, 4:7]
            axis = vmfs[:, 1:4]

            h, w = rhs_image.shape[0], rhs_image.shape[1]

            fig, ax = plt.subplots(1, 2, figsize=(12, 6))

            ax[0].imshow(np.clip(rhs_image ** (1.0 / 2.2), 0, 1))
            ax[0].axis("off")
            ax[0].set_title(f"RHS Image - epoch:{epoch}")

            plot_vapl_means(ts.scene, mean, variance, amplitude,axis, h, w, ax[1])
            ax[1].imshow(np.clip(rhs_image ** (1.0 / 2.2), 0, 1))
            ax[1].axis("off")
            ax[1].set_title(f"RHS + VAPL - epoch:{epoch}")

            plt.show()